In [1]:
import sys
sys.executable, sys.version

('/Users/ozkansafak/code/retrieval/.venv/bin/python',
 '3.10.9 (main, Mar  1 2023, 12:20:14) [Clang 14.0.6 ]')

In [2]:
%load_ext autoreload
%autoreload 2
# %matplotlib inline

from utils.utils import *
from utils.tt_helpers import *
from sentence_transformers import SentenceTransformer
os.environ["TOKENIZERS_PARALLELISM"] = "false"  # Suppress huggingface warning

sbert = SentenceTransformer('paraphrase-MiniLM-L6-v2')


ModuleNotFoundError: No module named 'mutils'

### Get Embeddings

In [18]:
# Load data
queries, query_embeddings, product_embeddings = load_raw_queries()

# load ids_toys from disk
with open('output/ids_toys.pkl', 'rb') as file:
    ids_toys = pickle.load(file)
    print(len(ids_toys))
    ids_present = [_id for _id in ids_toys if _id in query_embeddings]

# Assemble qb & xb input embeddings
qb = torch.stack([torch.tensor(query_embeddings[_id]) for _id in ids_present])
xb = torch.tensor(np.array([product_embeddings[_id] for _id in ids_present]))
qb.shape, xb.shape

### Split Dataset into Train and Test Sets
(qb_train, qb_test), (xb_train, xb_test), (idx_train, idx_test) =  shuffle_and_split(qb, xb)
print('qb_train.shape:', qb_train.shape)
print('qb_test.shape:', qb_test.shape)


### Two-Tower Pytorch Model

In [21]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class TwoTowerNetwork(nn.Module):
    def __init__(self, d, hidden_dim):
        super(TwoTowerNetwork, self).__init__()

        # qb_layer: query embedding input layer
        self.qb_layer = nn.Sequential(
            nn.Linear(d, hidden_dim),
            nn.ReLU(),
            nn.Linear(d, hidden_dim),
        )

        # xb_layer: product embedding input layer
        self.xb_layer = nn.Sequential(
            nn.Linear(d, hidden_dim),
            nn.ReLU(),
            nn.Linear(d, hidden_dim),            
        )
        
        self.softmax = nn.Softmax(dim=-1)

    def forward(self, u, v):
        qb_output = self.qb_layer(u)
        xb_output = self.xb_layer(v)

        dot_product = torch.einsum ('ik, jk -> ij', qb_output, xb_output)

        output = self.softmax(dot_product)

        return output

d = hidden_dim = qb.shape[-1]

# Initialize the network
model = TwoTowerNetwork(d, hidden_dim)

# Forward pass through the model
# output = model(qb_train, xb_train)


In [22]:
## TRAIN
import torch
import torch.optim as optim
import torch.nn as nn

# Hyperparameters
learning_rate = 0.0001
num_epochs = 200
batch_size = 32

# Define the loss function and the AdamW optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

targets_batch = torch.arange(batch_size, dtype=torch.long)
# Training loop
for epoch in range(num_epochs):
    for start in range(0, len(qb_train), batch_size):
        if start + batch_size > len(qb_train):
            continue
        qb_batch = qb_train[start:start+batch_size]
        xb_batch = xb_train[start:start+batch_size]

        # Forward pass: compute predictions
        outputs = model(qb_batch, xb_batch)

        # Compute the loss
        loss = criterion(outputs, targets_batch)

        # Backward pass and optimization
        optimizer.zero_grad()  # zero out the gradient ops
        loss.backward()  # compute gradients
        optimizer.step()  # apply gradients on the parameters.

    print(f"Epoch [{epoch+1:2d}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training complete.")


Epoch [ 1/200], Loss: 1.0377
Epoch [ 2/200], Loss: 0.8360
Epoch [ 3/200], Loss: 0.7469
Epoch [ 4/200], Loss: 0.6681
Epoch [ 5/200], Loss: 0.6128
Epoch [ 6/200], Loss: 0.5852
Epoch [ 7/200], Loss: 0.5720
Epoch [ 8/200], Loss: 0.5658
Epoch [ 9/200], Loss: 0.5623
Epoch [10/200], Loss: 0.5600
Epoch [11/200], Loss: 0.5584
Epoch [12/200], Loss: 0.5573
Epoch [13/200], Loss: 0.5564
Epoch [14/200], Loss: 0.5558
Epoch [15/200], Loss: 0.5552
Epoch [16/200], Loss: 0.5548
Epoch [17/200], Loss: 0.5545
Epoch [18/200], Loss: 0.5542
Epoch [19/200], Loss: 0.5539
Epoch [20/200], Loss: 0.5537
Epoch [21/200], Loss: 0.5535
Epoch [22/200], Loss: 0.5534
Epoch [23/200], Loss: 0.5532
Epoch [24/200], Loss: 0.5531
Epoch [25/200], Loss: 0.5530
Epoch [26/200], Loss: 0.5529
Epoch [27/200], Loss: 0.5528
Epoch [28/200], Loss: 0.5528
Epoch [29/200], Loss: 0.5527
Epoch [30/200], Loss: 0.5526
Epoch [31/200], Loss: 0.5526
Epoch [32/200], Loss: 0.5525
Epoch [33/200], Loss: 0.5525
Epoch [34/200], Loss: 0.5524
Epoch [35/200]

### Query all `_id`s from ES

In [32]:
ids_list = query_all_ids_from_es(index, elSearch)

2000 Runtime: 0 min 1 sec
3000 Runtime: 0 min 1 sec
4000 Runtime: 0 min 2 sec
5000 Runtime: 0 min 2 sec
6000 Runtime: 0 min 3 sec
7000 Runtime: 0 min 3 sec
8000 Runtime: 0 min 3 sec
9000 Runtime: 0 min 3 sec
10000 Runtime: 0 min 3 sec
11000 Runtime: 0 min 4 sec
12000 Runtime: 0 min 4 sec
13000 Runtime: 0 min 4 sec
14000 Runtime: 0 min 4 sec
15000 Runtime: 0 min 4 sec
16000 Runtime: 0 min 5 sec
17000 Runtime: 0 min 5 sec
18000 Runtime: 0 min 5 sec
19000 Runtime: 0 min 5 sec
20000 Runtime: 0 min 6 sec
21000 Runtime: 0 min 6 sec
22000 Runtime: 0 min 6 sec
23000 Runtime: 0 min 6 sec
24000 Runtime: 0 min 6 sec
25000 Runtime: 0 min 6 sec
26000 Runtime: 0 min 7 sec
27000 Runtime: 0 min 7 sec
28000 Runtime: 0 min 7 sec
29000 Runtime: 0 min 7 sec
30000 Runtime: 0 min 7 sec
31000 Runtime: 0 min 8 sec
32000 Runtime: 0 min 8 sec
33000 Runtime: 0 min 8 sec
34000 Runtime: 0 min 8 sec
35000 Runtime: 0 min 8 sec
36000 Runtime: 0 min 9 sec
37000 Runtime: 0 min 9 sec
38000 Runtime: 0 min 9 sec
39000 Run

In [125]:
## load ids_all from disk
# with open('output/ids_all.pkl', 'rb') as file:
#     ids_all = pickle.load(file)

## load ids_toys from disk
# with open('output/ids_toys.pkl', 'rb') as file:
#     ids_toys = pickle.load(file)


In [ ]:
# Compute embeddings out of SBERT

# INPUTS
load = True

start = time.time()
from dialog_agent_service.search.ElSearch import elSearch

if load:
    queries, query_embeddings, product_embeddings = load_raw_queries()
else:
    product_embeddings = dict()
    queries = dict()
    query_embeddings = dict()
    
istart = len(queries)
print(f'istart: {istart}')

for i, _id in enumerate(ids_toys[istart:], start=istart):
    product_info = query_by_id(_id)
    name, details, description = get_product_info_from_dict(product_info)
    sentence = f"""Product Name:"{name}", Product Details:"{details}", Product Description:"{description}" """
    
    product_embeddings[_id] = sbert.encode(sentence)
    queries[_id] = generate_gpt_queries(name, details, description)
    query_embeddings[_id] = sbert.encode(queries[_id])

    print('.', end='')
    if (i+1) % 100 == 0:
        print(f" i:{i} {print_runtime(start, False)}")
        write_raw_queries(queries, query_embeddings, product_embeddings)


loaded (queries, query_embeddings, product_embeddings) from  output/raw_queries.pkl
istart: 21000
.................................................................................................... i:21099 Runtime: 7 min 4 sec
written to output/raw_queries.pkl: Runtime: 0 min 0 sec
.................................................................................................... i:21199 Runtime: 13 min 26 sec
written to output/raw_queries.pkl: Runtime: 0 min 0 sec
.................................................................................................... i:21299 Runtime: 20 min 0 sec
written to output/raw_queries.pkl: Runtime: 0 min 0 sec
.................................................................................................... i:21399 Runtime: 26 min 28 sec
written to output/raw_queries.pkl: Runtime: 0 min 0 sec
.................................................................................................... i:21499 Runtime: 33 min 18 sec
written to output/raw

### query `name`, `details` and `description` and store in a local json file

In [ ]:
# run this in a cell of its own
info = dict()


In [164]:
start = time.time()

istart = len(info)
print(f'istart: {istart}')

for i, _id in enumerate(ids_toys[istart:], start=istart):
    product_info = query_by_id(_id)
    name, details, description = get_product_info_from_dict(product_info)
    
    info[_id] = dict()
    info[_id]['name'] = name
    info[_id]['details'] = details
    info[_id]['description'] = description
    print('.', end='')
    if i % 1000 == 0:
        print(i)
        print_runtime(start)
        
with open('output/product_info.json', 'w') as f:
    json.dump(info, f)

print_runtime(start)


istart: 23865
........................................................................................................................................24000
Runtime: 0 min 8 sec
.......................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

In [165]:
with open('output/product_info.json', 'w') as f:
    json.dump(info, f)


In [ ]:
# stack up a list of tensors into single query (qb) and product (xb) tensors.
def construct_qb_xb(query_embeddings, product_embeddings, ids_toys):
    qb = torch.stack([torch.tensor(query_embeddings[_id]) for _id in ids_toys])
    xb = torch.tensor(np.array([product_embeddings[_id] for _id in ids_toys]))
    
    # Save tensors and lists and dicts
    torch.save({'qb': qb, 'xb': xb}, 'output/qb_xb.pt')
    write_raw_queries(queries, query_embeddings, product_embeddings, pids)
    
    # Double check if the shapes look correct.
    print(qb.shape, xb.shape, qb.requires_grad, xb.requires_grad)
    print(qb, xb)

qb, xb = construct_qb_xb(query_embeddings, product_embeddings, ids_toys)


---
###

good for gift: boy, baby
good for gift: boy, 1 year
good for gift: boy, 2 years
good for gift: boy, 3 years
good for gift: boy, 4 years
good for gift: boy, 5 years
good for gift: boy, 6 years
good for gift: boy, 7 years
good for gift: boy, 8 years
good for gift: boy, 9 years
good for gift: boy, 10 years
good for gift, boy, 11 years
good for gift: boy, 12 years
good for gift: girl, baby
good for gift: girl, 1 year
good for gift: girl, 2 years
good for gift: girl, 3 years
good for gift: girl 4 years
good for gift: girl, 4 years
good for gift: girl, 5 years
good for gift: girl, 6 years
good for gift: girl, 7 years
good for gift: girl, 8 years
good for gift: girl, 9 years
good for gift, girl 9 years
good for gift, girl, 10 years
good for gift: girl, 10 years
good for gift: girl 11 years
good for gift: girl 12 years
good for age: 3
good for age: 4
good for age: 5
good for age: 6
good for age: 7
good for age: 8
good for age: 9
good for age: 10
good for age: 11
good for age: 12
good for gender: boy
good for gender: girl
good for gender: unisex
good for gift: overall
good for ocassion: birthday
good for ocassion: christmas
good for ocassion: hannukah
good for ocassion: holiday


### Serving from the Faiss index


In [1]:
# import faiss
# # train the faiss index (Voronai cells + product quantizer)
# index = faiss.IndexFlatL2(product_embeddings[pid].shape[0])  # build the index
# index.add(np.array(xb))  # add vectors to the index

# # search 
# k = 4  # capture 4 nearest neighbors
# D, I = index.search(xb[:5], k=k)  # sanity check
# print(I)
# print(D)
# D, I = index.search(xq, k=k) # actual search
